In [8]:
# import important library
import pandas as pd
import numpy as np
import bot

In [125]:
import importlib
importlib.reload(bot)

<module 'bot' from '/Users/henrytran/Documents/GitHub/aias_trading25/bot.py'>

# 1. Data Processing

In [10]:
# read the csv file
daily_df=pd.read_csv('BTC-Daily.csv')
daily_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [11]:
# convert time strings into timestamps, and take only year, month and date values
daily_df["date"]=pd.to_datetime(daily_df["date"]).dt.strftime("%Y-%m-%d")

In [12]:
daily_df.shape

(2651, 9)

The dataset has 2651 records from 2014 to 2022.

As the data is needed for training and testing, every records before 2020 will be used for training and every records after 2020 will be used for testing.

In [130]:
# splitting the data
training = daily_df[daily_df["date"]<"2020-01-01"].iloc[::-1]
testing = daily_df[daily_df["date"]>="2020-01-01"].iloc[::-1]

In [131]:
# convert the data into csv files
training.to_csv('training.csv')
testing.to_csv('testing.csv')

After splitting, the training dataset has 1860 records and the testing data has 791 records.

In [15]:
print(training.shape)
print(testing.shape)

(1860, 9)
(791, 9)


# 2. Hill-climbing algorithm

## a. Try the bot

In [16]:
# extract the data
close_price_train=training["close"]
no_of_day_train=training.shape[0]
close_price_test =testing["close"]
no_of_day_test=testing.shape[0]

In [17]:
low_sma = bot.wma(close_price_train, 10, bot.sma_filter(10))
high_sma = bot.wma(close_price_train, 20, bot.sma_filter(20))

print(low_sma)
print(len(low_sma))
print(high_sma)
print(len(high_sma))

[-5844.114 -4371.12  -2900.949 ...   371.484   374.107   375.274]
1860
[-6466.681  -5746.248  -5014.0975 ...   354.9755   357.9115   361.117 ]
1860


In [18]:
# self-implementation for trading bot of sma
def bot_sma(close_price, hfw, lfw):
    low_sma = bot.wma(close_price, lfw, bot.sma_filter(lfw))
    high_sma = bot.wma(close_price, hfw, bot.sma_filter(hfw))
    buy_sell_signals=[]
    for i in range(len(low_sma)-1):
        if low_sma[i]<high_sma[i] and low_sma[i+1]>high_sma[i+1]:
            buy_sell_signals.append("buy")
        elif low_sma[i]>high_sma[i] and low_sma[i+1]<high_sma[i+1]:
            buy_sell_signals.append("sell")
        else:
            buy_sell_signals.append("-")
    return buy_sell_signals

## b. Hill-climbing algorithm

In [165]:
def hill_climbing(bot_type, high_frequency_window, low_frequency_window, alpha=0, max_iter=100):
    high_window = high_frequency_window
    low_window = low_frequency_window
    new_high_frequency_window = high_frequency_window
    new_low_frequency_window= low_frequency_window
    alpha=alpha
    new_alpha=alpha
    cash1=0
    for i in range(max_iter):
        # find the correct parameter tweak:
        if bot_type.lower() == 'sma':
            new_high_frequency_window, new_low_frequency_window = parameter_tweak('sma', high_window, low_window)
        elif bot_type.lower() == 'smaema' and alpha !=0:
            new_high_frequency_window, new_low_frequency_window, new_alpha = parameter_tweak('smaema', low_window, high_window, alpha)
        elif bot_type.lower() == 'complex':
            new_high_frequency_window, new_low_frequency_window = parameter_tweak('complex',high_window, low_window)
            
        # run the total cash return after trading
        cash1=bot_fitness_func(bot_type, high_window, low_window, alpha)
        cash2=bot_fitness_func(bot_type, new_high_frequency_window, new_low_frequency_window, new_alpha)
        # compare the cash earned after tweaking the parameters
        if cash2 > cash1:
            high_window = new_high_frequency_window
            low_window = new_low_frequency_window
            alpha=new_alpha
            cash1 = cash2
    if bot_type.lower() == 'sma' or bot_type.lower() == 'complex':
        return high_window, low_window, cash1
    elif bot_type.lower() == 'smaema':
        return high_window, low_window, alpha, cash1

def parameter_tweak(bot_type, hfw, lfw, alpha =0):
    if bot_type.lower() == 'sma':
        new_hfw, new_lfw = window_tweak(hfw, lfw)
        return new_hfw, new_lfw
    elif bot_type.lower() == 'smaema' and alpha !=0:
        new_hfw, new_lfw = window_tweak(hfw, lfw)
        new_alpha = alpha_tweak(alpha)
        return new_hfw, new_lfw, new_alpha
    elif bot_type.lower() == 'complex':
        new_hfw, new_lfw = complex_tweak(hfw, lfw)
        return new_hfw, new_lfw

def window_tweak(hfw, lfw):
    rng = np.random.default_rng()
    for _ in range(100):
        # new high frequency window
        a=rng.integers(-5,6)
        new_hfw=hfw+a
        # new low frequency window
        b=rng.integers(-5,6)
        new_lfw=lfw+b
        # we check to make sure that new_hfw in range(11,40) and new_lfw(2,10)
        new_hfw = max(2, max(2, new_hfw))
        new_lfw = max(11, min(40, new_lfw))
        if new_hfw < new_lfw:
            return new_hfw, new_lfw
    return hfw, lfw

def alpha_tweak(alpha):
    rng = np.random.default_rng()
    for _ in range(100):
        diff = rng.uniform(-0.15, 0.15)
        new_alpha = alpha + diff
        if 0.1<= new_alpha <= 1:
            return new_alpha
    return alpha # if after 100 loops and cannot find the optimal value, then we return the original alpha

def weights_tweak(weight_lst):
    rng = np.random.default_rng()
    # random index number of a weight in a list
    position=rng.integers(0,3)
    # normalised weights
    normalised_weights = [float(weight)/sum(weight_lst) for weight in weight_lst]

    # tweak the randomly selected weight with proportion different
    selected_weight_rate = normalised_weights[position]
    diff = np.random.uniform(-0.15, 0.15)
    selected_weight_rate += diff

    # the rate of other weights
    other_weight_idx= [idx for idx in range(3) if idx!=position]
    remainder_rate = 1- selected_weight_rate 
    # the remaining 2 weights, we generate randomly the proportion and multiply it with the current rates 
    one_weight_rate = rng.uniform(0.1, 0.9)
    another_weight_rate = 1- one_weight_rate
    other_weight_rates=[one_weight_rate, another_weight_rate]
    new_other_weight_rates = [remainder_rate * s for s in other_weight_rates]

    new_weights = [0,0,0]
    new_weights[position] = selected_weight_rate * sum(weight_lst)
    for i in range(2):
        idx = other_weight_idx[i]
        new_weights[idx] = new_other_weight_rates[i] * sum(weight_lst)

    return new_weights[0], new_weights[1], new_weights[2]


def complex_tweak(hfw, lfw):
    # in this code, hfw and lfw input are array-type in format of [w1, w2, w3, d1, d2, d3, sf]
    # weight tweak
    new_hfw_w1, new_hfw_w2,new_hfw_w3 = weights_tweak(hfw[0:3])
    new_lfw_w1, new_lfw_w2,new_lfw_w3 = weights_tweak(lfw[0:3])

    # window tweak
    new_hfw_d1, new_lfw_d1 = window_tweak(hfw[3], lfw[3])
    new_hfw_d2, new_lfw_d2 = window_tweak(hfw[4], lfw[4])
    new_hfw_d3, new_lfw_d3 = window_tweak(hfw[5], lfw[5])

    # alpha tweak
    new_hfw_alpha = alpha_tweak(hfw[-1])
    new_lfw_alpha = alpha_tweak(lfw[-1])

    return [new_hfw_w1, new_hfw_w2, new_hfw_w3, new_hfw_d1, new_hfw_d2, new_hfw_d3,new_hfw_alpha], [new_lfw_w1, new_lfw_w2, new_lfw_w3, new_lfw_d1, new_lfw_d2, new_lfw_d3, new_lfw_alpha]



In [156]:
#sma
hfw_result, lfw_result, cash_result = hill_climbing('sma',5,30)
hfw_result, lfw_result, cash_result

(np.int64(9), np.int64(25), np.float64(5965.086841833178))

In [166]:
#smaema
hfw_result, lfw_result, alpha, cash_result = hill_climbing('smaema', 30, 5, 0.2)
hfw_result, lfw_result, alpha, cash_result

(2, np.int64(25), 0.1273048476865516, np.float64(878.8118023812054))

In [143]:
#complex
hfw_result, lfw_result, cash_result = hill_climbing('complex',[1,2,3,5,5,5,0.3],[3,2,1,30,30,30,0.1])
hfw_result, lfw_result, cash_result

([1.1165307812285363,
  1.425014161644454,
  3.45845505712701,
  np.int64(10),
  np.int64(19),
  np.int64(14),
  0.4775543721122697],
 [2.549860289112541,
  0.3703404855701279,
  3.0797992253173296,
  np.int64(25),
  np.int64(24),
  np.int64(23),
  0.22189750855758114],
 np.float64(34760.78854586701))

# 3. Fitness function and Bot evaluation function

In [159]:
def bot_fitness_func(bot_type, high_window, low_window, alpha=0): # change bot_signals to high_frequency_window_size, low_frequency_window_size
    """
    This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot
    Parameters:
    - 
    - 
    Return:
    - 
    """
    # intialise bot, use training dataset for optimisation algorithms fitness functions
    bot_signals=[]
    close_price = pd.read_csv('training.csv')['close']
    
    if bot_type.lower() == 'sma' and alpha ==0:
        bot_signals = bot.get_signals_sma2(close_price, high_window, low_window) #need to think about how to call 2 other bot algorithms
    elif bot_type.lower() == 'smaema' and alpha!=0:
        bot_signals = bot.get_signals_smaema(close_price, low_window, high_window, alpha)
    elif bot_type.lower() == 'complex' and alpha==0: #as alpha values were stored in the list of high_window and low_window
        bot_signals = bot.get_signals_complex(close_price, high_window, low_window)

    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    #loop through the time length
    for i in range(len(close_price)-1):
        close=close_price.iloc[i]
        # buy, ensure we have cash to buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
        # sell, ensure we have bitcoin to sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
    
    return cash

In [145]:
def bot_testing(bot_type, high_window, low_window, alpha=0):
    # This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot, including the time of the transactions made
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    # get the data
    close_price = pd.read_csv('testing.csv')['close']
    time = pd.read_csv('testing.csv')['date']

    # the list to save the transaction history
    result=[]
    bot_signals=[]

    # intialise bot, use training dataset for optimisation algorithms fitness functions
    if bot_type.lower() == 'sma' and alpha ==0:
        bot_signals = bot.get_signals_sma2(close_price, high_window, low_window)
    elif bot_type.lower() == 'smaema' and alpha!=0:
        bot_signals = bot.get_signals_smaema(close_price, low_window, high_window, alpha)
    elif bot_type.lower() == 'complex' and alpha==0: #as alpha values were stored in the list of high_window and low_window
        bot_signals = bot.get_signals_complex(close_price, high_window, low_window)
    
    #loop through the time length
    for i in range(time.shape[0]-1):
        close=close_price[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close
            cash = 0
            result.append([time[i],cash, bitcoin])
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close * (1-fee)
            bitcoin =0
            result.append([time[i],cash, bitcoin])
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
        result.append([time[i],cash, bitcoin])
    
    return result

In [146]:
def bot_evaluation(bot_type, high_window, low_window, alpha=0):
    # This function will returns the result nicely
    result_lst=bot_testing(bot_type, high_window, low_window, alpha)
    result_df=pd.DataFrame(result_lst, columns=["Time", "Cash", "Bitcoin"])
    print(result_df.to_string(index=False, justify="center", float_format='{:,.2f}'.format))
    return result_df

In [150]:
#sma - nice printout
bot_evaluation('sma', 5, 30, alpha=0)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-02-25 1,219.65   0.00  
2020-04-04     0.00   0.17  
2020-05-25 1,485.17   0.00  
2020-05-30     0.00   0.15  
2020-06-15 1,358.53   0.00  
2020-07-10     0.00   0.14  
2020-07-11 1,272.01   0.00  
2020-07-12     0.00   0.13  
2020-07-13 1,187.44   0.00  
2020-07-22     0.00   0.12  
2020-08-26 1,343.28   0.00  
2020-09-20     0.00   0.12  
2020-09-21 1,204.63   0.00  
2020-09-28     0.00   0.11  
2021-01-23 3,405.50   0.00  
2021-02-04     0.00   0.09  
2021-04-21 4,660.14   0.00  
2021-05-09     0.00   0.08  
2021-05-12 3,714.01   0.00  
2021-06-14     0.00   0.09  
2021-06-20 3,068.35   0.00  
2021-07-25     0.00   0.08  
2021-09-10 3,655.38   0.00  
2021-10-03     0.00   0.07  
2021-11-18 4,058.38   0.00  
2021-12-27     0.00   0.08  
2021-12-28 3,579.50   0.00  
2022-02-06     0.00   0.08  
2022-02-21 2,943.69   0.00  


,Time,Cash,Bitcoin
0,2020-01-01,0.000000,0.135122
1,2020-02-25,1219.646350,0.000000
2,2020-04-04,0.000000,0.171966
3,2020-05-25,1485.173733,0.000000
4,2020-05-30,0.000000,0.148439
5,2020-06-15,1358.530905,0.000000
6,2020-07-10,0.000000,0.141948
7,2020-07-11,1272.014046,0.000000
8,2020-07-12,0.000000,0.132584
9,2020-07-13,1187.440707,0.000000


In [158]:
#smaema
bot_evaluation('smaema',30,5, 0.2)

   Time      Cash    Bitcoin
2020-01-26     0.00   0.11  
2020-01-29 1,017.43   0.00  
2020-02-19     0.00   0.10  
2020-03-08   801.47   0.00  
2020-03-10     0.00   0.10  
2020-03-23   620.95   0.00  
2020-03-31     0.00   0.09  
2020-04-03   613.50   0.00  
2020-04-14     0.00   0.09  
2020-04-19   597.93   0.00  
2020-04-24     0.00   0.08  
2020-04-25   565.75   0.00  
2020-05-13     0.00   0.06  
2020-05-16   536.52   0.00  
2020-05-24     0.00   0.06  
2020-05-30   561.91   0.00  
2020-06-14     0.00   0.06  
2020-06-24   526.22   0.00  
2020-06-27     0.00   0.06  
2020-07-08   518.63   0.00  
2020-07-17     0.00   0.05  
2020-07-22   508.39   0.00  
2020-08-23     0.00   0.04  
2020-09-01   489.82   0.00  
2020-09-04     0.00   0.05  
2020-09-16   482.70   0.00  
2020-09-24     0.00   0.04  
2020-09-28   451.86   0.00  
2020-10-04     0.00   0.04  
2020-10-07   425.14   0.00  
2020-11-29     0.00   0.02  
2020-12-02   422.76   0.00  
2020-12-11     0.00   0.02  
2020-12-15   4

,Time,Cash,Bitcoin
0,2020-01-26,0.000000,0.112916
1,2020-01-29,1017.427369,0.000000
2,2020-02-19,0.000000,0.102848
3,2020-03-08,801.465676,0.000000
4,2020-03-10,0.000000,0.098584
...,...,...,...
71,2022-01-16,138.772464,0.000000
72,2022-01-17,0.000000,0.003186
73,2022-02-01,119.687701,0.000000
74,2022-02-20,0.000000,0.003025


In [152]:
#complex
bot_evaluation('complex',[1,2,3,5,5,5,0.3],[3,2,1,30,30,30,0.1])

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-01-30 1,245.15   0.00  
2020-04-09     0.00   0.17  
2020-04-11 1,105.59   0.00  
2020-04-30     0.00   0.12  
2020-05-12 1,064.14   0.00  
2020-07-30     0.00   0.09  
2020-08-09 1,052.69   0.00  
2020-10-24     0.00   0.08  
2020-10-29 1,016.57   0.00  
2020-10-31     0.00   0.07  
2020-11-01   952.75   0.00  
2020-11-05     0.00   0.06  
2020-11-26   985.77   0.00  
2020-12-18     0.00   0.04  
2021-01-19 1,439.91   0.00  
2021-02-08     0.00   0.03  
2021-02-25 1,374.43   0.00  
2021-03-13     0.00   0.02  
2021-03-15 1,176.48   0.00  
2021-07-29     0.00   0.03  
2021-08-04 1,098.75   0.00  
2021-08-07     0.00   0.02  
2021-08-18 1,036.32   0.00  
2021-10-07     0.00   0.02  
2021-10-24 1,104.20   0.00  


,Time,Cash,Bitcoin
0,2020-01-01,0.000000,0.135122
1,2020-01-30,1245.152312,0.000000
2,2020-04-09,0.000000,0.165589
3,2020-04-11,1105.588426,0.000000
4,2020-04-30,0.000000,0.124305
5,2020-05-12,1064.139088,0.000000
6,2020-07-30,0.000000,0.092891
7,2020-08-09,1052.689846,0.000000
8,2020-10-24,0.000000,0.077812
9,2020-10-29,1016.569335,0.000000
